<div width=50% style="display: block; margin: auto">
    <img src="images/ic_logo.png" width="200">
    <img src="images/sberbank_logo.png" width="250" style="margin-left: 10%">
</div>

<hr width=55% style="float: left">

#### День 3
# Компьютерное зрение. Масштабируемое распознавание с помощью словарного дерева

### [Antonio Stanziola](https://github.com/astanziola) и [Eduardo Pignatelli](https://github.com/epignatelli)

<hr width=70% style="float: left">

<div class="card">
    <div class="card-header"><h4> ✔️ Цели</h4></div>
</div>

Целью этого практического занятия является реализация системы поиска изображений, похожих на данное изображение. В процессе работы вы узнаете, как изображение можно описать как векторы, а также как эффективно искать похожие векторы в большой базе данных

<div class="panel panel-default">
  <div class="panel-heading"><h4> 📜 Содержание</h4></div>
  <div class="panel-body">

1. [Вступление](#1.-Introduction)
  - Задача поиска изображений
  - Конвейер CBIR
2. [Извлечение признаков](##2.-Features-extraction)
  - Поиск ключевых точек
  - Детекторы углов
  - Извлечение локальных признаков
3. [Создание визуального словаря](##3.-Creating-a-visual-vocabulary)
  - Зачем использовать словарь?
  - Построение дерева с помощью иерархических k-средних
  - Индексирование базы данных с помощью TF-IDF (частота слова и обратная частота документа)
4. [Оценка и поиск (сетевой этап)](#4-Scoring-and-Retrieving-(the-online-phase))
  - Поиск изображения в базе данных
5. [Кодирование изображений с помощью глубинных сверточных нейронных сетей](#5.Image-encoding-with-Deep-Convolutional-Neural-Networks)
<p/>
  </div>
</div>

Пока мы представляем задачу, давайте скачаем данные, которые будем использовать в течение всего занятия. 

In [ ]:
!python cbir/download.py

<br>

## **1. Вступление**
---

### 1.1 Задача поиска изображений

<img src="images/CBIR.png" height="10" width="400" align="right">

Поиск изображений по содержанию (англ. Content-based image retrieval или CBIR) — это процесс поиска изображений в большой базе данных при наличии визуального поискового запроса. С технической точки зрения есть три компонента CBIR:  
  1. Представление изображений
  2. Организация базы данных
  3. Измерение расстояния между изображениями

Определение CBIR можно описать подробнее.

> CIBR использует ***представление*** виртуального содержимого для выявления связанных изображений в базе данных.

<u>В этом уроке мы сосредоточимся на задаче **поиска изображений по запросу**, где в качестве запроса используется образец изображения.</u>

В конце этого урока должна получится указанная ниже функция.  
Из этого пособия вы узнаете все составляющие, необходимые для реализации даннойфункции.

In [ ]:
def retrieve(database, query):
    raise NotImplemented

<div class="alert alert-heading alert-danger" style="background-color: white; border: 2px solid; border-radius: 5px; color: #000; border-color:#AAA; padding: 10px">
<b>📝 Примечание </b>

**На этом занятии и воспроизведем и реализуем метод из работы, которая стала основой для Поиска Google по изображениям:**  
***[Nister, D. and Stewenius, H., 2006 г., июнь. Масштабируемое распознавание с помощью словарного дерева. Материалы конференции компьютерного сообщества IEEE 2006 года по вопросам компьютерного зрения и распознавания шаблонов (CVPR'06) (том. 2, с. 2161-2168). Ieee.](https://www.imperial.ac.uk/media/imperial-college/faculty-of-engineering/bioengineering/public/directions/Directions-to-Bessemer-Level-4-Meeting-Rooms-1-and-2.pdf)***

</div>

### 1.2 Процесс CBIR
<hr width=20% align=left>
Можно выделить два основных этапа CBIR: **автономный** этап и **сетевой** этап [ссылка]. Из этих этапов состоит высокоуровневая архитектура. Отдельно взятые компоненты используют комбинацию техник (например, обучение с учителем и без учителя).

<div style="img {align: left}">
<img src="images/pipeline_extended.png">
<em>Источник рис. — Zheng и др, 2017 г. «SIFT и CNN: десятилетнее исследование поиска экземпляров».</em>
<p/>
</div>


Целью **автономного** этапа является исопльзование стека изображений для создания индексированной базы данных. Мы стараемся делать так, чтобы основная вычислительная нагрузка приходилось на этот этап, чтобы упростить сетевой этап.
Используются две основные операции:
- Создание представления изображений
- Эффективное индексирование изображений



На **сетевом** этапе стоит задача — после получения изображения-*запроса* оценить часть изображений или все изображения в базе данных, а затем вернуть изображения с наивысшими баллами.



Приведем пример реализации простой системы CBIR. Мы разделим CBIR на функции и рассмотрим каждый функциональный блок по мере продвижения в пособии. В последней части занятия мы соберем все блоки вместе.

In [ ]:
class CBIR:
    def __init__(self, dataset):
        self.dataset = dataset
        self.database = None  # индексированная база данных еще не собрана
        return
    
    #-- ЗАДАЧА 1
    def find_keypoints(self, image):
        # реализуйте с помощью MSER
        raise NotImplemented
        
    #-- ЗАДАЧА 2
    def extract_features(self, image):
        # реализуйте с помощью SIFT
        raise NotImplemented
    
    #-- ЗАДАЧА 3
    def create_vocabulary(self, image):
        # реализуйте с помощью extract_features и BOW
        raise NotImplemented

    #-- ЗАДАЧА 4
    def encode(self, dataset):
        # реализуйте с помощью словарного дерева
        raise NotImplemented
    
    #-- ЗАДАЧА 5
    def score(self, ref_image, query_image):
        raise NotImplemented
    
    #-- ЗАДАЧА 6
    def retrieve(self, query):
        # это описанная выше функция
        raise NotImplemented

<br>

## **2. Извлечение признаков**


### 2.1 Поиск ключевых точек
<hr width=20% align=left>

<div style="margin:auto; float:right; margin-left: 50px; width: 45%">
<img src="images/features_extraction_only.jpeg">
<em>Разложение изображения на несколько «отличительных» локализованных регионов</em>
</div>

В этом разделе мы узнаем, как реализовать функцию:
```python
def find_keypoints(self, image):
    raise features
```
Она извлекает набор ключевых точек, используемых для описания ихображения.

Ключевые точки можно представить как **ориентиры** в сцене. Как и ориентиры в «человеческом» понимании, ключевые точки представляют собой определенные места в пространстве. В отличие от выбираемых людьми ориентиров, контрольные точки можно рассчитать непосредственно из данных изображения на основе такого принципа: ориентиры должны быть относительно разреженными, отличительными и стабильными. Когда ориентиры найдены, их можно использовать в качестве справочных точек, вокруг которыъ будут строиться дескрипторы окружающей структуры изображения.

#### Операции с изображениями

Чтобы быстро производить операции с изображениями мы будем использовать [библиотеку open-cv](https://docs.opencv.org/master/d6/d00/tutorial_py_root.html). Сначала импортируем ее вместе с библиотекой `numpy` и `matplotlib` для построения изображений.

Мы также вызываем функции, которые обрабатывают набор данных изображения, выполняют операции с ними и показывают изображения, а также отображают точки-ориентиры, которые мы обнаружим.

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

import cbir

# Загрузка набора данных
dataset = cbir.Dataset()
descriptor = cbir.descriptors.Orb()

Рассмотрим случайное изображение из набора данных:

In [ ]:
img = dataset.get_random_image()

# Построение изображения
plt.figure(figsize=(10,10))
dataset.show_image(img)
plt.title("Случайное изображение из набора данных")
plt.show()

### Детектор углов 

Возможный набор интересных точек — это углы объектов на изображении. Мы будем использовать оператор детектора углов, разработанного [Гаррисом и Стивенсом](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_features_harris/py_features_harris.html), для их извлечения с изображения.

Начнем с загрузки изображения с четко определенными углами в оттенках серого.
Мы также можем легко выполнять операции с изображением с помощью `numpy`, как на примере ниже.

In [ ]:
# Загрузка и отображение изображения
img = dataset.read_image('110901.jpg')
z = np.sin(img/10.)

# Функции построения
plt.figure(figsize=(12,4))
plt.subplot(121)
dataset.show_image(img, gray=True)
plt.title('110901.jpg')
plt.subplot(122)
dataset.show_image(z, gray=True)
plt.title("$\sin(I/10)$")
plt.show()

Реализация детектора в opencv находится в `cv2.cornerHarris()` ([документация](https://docs.opencv.org/2.4/modules/imgproc/doc/feature_detection.html?highlight=cornerharris#cornerharris)). Попробуйте настроить параметры функций, чтобы улучшить обнаружение углов. Также поэкспериментируйте с разными изображениями.

<div class="alert alert-block alert-info">
<b>📝 Упражнение</b>

Попробуйте изменить изображения выше и параметры обнаружения углов, чтобы посмотреть, как это повлияет на результаты. Просмотрите [документацию](https://docs.opencv.org/2.4/modules/imgproc/doc/feature_detection.html?highlight=cornerharris#cornerharris) по функции `cv2.cornerHarris()`, чтобы подробнее узнать о значении каждого параметра.

</div>

In [ ]:
cbir.marking.this_is_an_action_cell()


# Параметры детектора углов

# --- Вставьте свой код сюда ---
block_size = 11
kernel_size = 11
aperture_parameter = 0.01
treshold = 0.001
# --- Конец кастомного кода ---


# --- Построение, не удаляйте код ниже ---
gray= cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
dst = cv2.cornerHarris(gray, 
                       blockSize=block_size, 
                       ksize=kernel_size, 
                       k=aperture_parameter)
dst = dst > treshold*dst.max()

plt.figure(figsize=(15,10))
descriptor.show_corners_on_image(gray,dst)

#### Лучшее обнаружение контрольных точек

Очевидно, на изображении кроме углов есть и другие интересные точки. Для них нужны более общие и утонченные детекторы ключевых точек. 

Пример современного детектора ключевых точек — метод [масштабно-инвариантной трансформации признаков (англ. scale-invariant feature transform, SIFT)](https://link.springer.com/content/pdf/10.1023/B:VISI.0000029664.99615.94.pdf). SIFT представляет собой дескриптор изобрадения для сопоставления и распознавания изображений. Его автором является Девид Лоу (1999, 2004). Этот дескриптор, а также связанные с ним дескрипторы изображений, используются для широкого спектра задач компьютерного зрения для сопоставления точек между разными проекциями 3D-сцены, а также распознавания объектов на основе проекций. На дескриптор SIFT практически не влияют преобразования, вращения и масштабирование в области отображения. Также он устойчив с умеренным изменениям перспективы и вариаций освещения [Лоу, 2004].

Еще один передовой детектор ключевых точек (и дескриптор, как мы увидим позже) представлен в [детекторе ORB (Oriented FAST and Rotated BRIEF](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_orb/py_orb.html), который представляет собой более быструю, но несколько менее точную версию SIFT.

В следующем коде вы настроите функцию детектора ORB, которую можно испытать на разных изображениях. Эту функцию можно использовать со всем набором данных, создавая список интересных точек для всех изображений. 

<div class="alert alert-block alert-info">
<b>📝 Упражнение</b>

Просмотрев [документацию по детектору ORB](https://docs.opencv.org/3.4/db/d95/classcv_1_1ORB.html#adc371099dc902a9674bd98936e79739c), особенно п ометодами `create()` и `detect()`, попробуйте реализовать функцию ниже. Она берет изображение в качестве вводных данных и выдает список [ключевых точек](https://docs.opencv.org/3.4/d2/d29/classcv_1_1KeyPoint.html), сгенерированный детектором ORB.

</div>

In [ ]:
cbir.marking.this_is_an_action_cell()


def find_keypoints(image):
    ''' Данная функция должна взять изображение в качестве входных данных
    и выдать список ключевых точек
    '''
    # --- Добавьте свой код сюда ---
    # Создание детектора и настройка свойств
    ...
    # --- Конец кастомного кода ---
    return keypoints

<details>
  <summary outline="1pt">🆘 Что-то не получается? Нажмите здесь, чтобы посмотреть возможное решение</summary>

```python
def find_keypoints(image):
    ''' Данная функция должна взять изображение в качестве входных данных
    и выдать список ключевых точек
    '''
    
    # Создание детектора и настройка свойств
    orb = cv2.ORB.create()
    
    # Обнаружение ключевых точек
    keypoints = orb.detect(image)
    
    return keypoints
```  
</details>

Теперь у нас есть функция для обнаружения ключевых точек, а также их масштаба и ориентации.

In [ ]:
img = dataset.read_image('110901.jpg')

keypoints = find_keypoints(img)

plt.figure(figsize=(15,10))
# Отображаются только каждые 10 ключевых точек
img2 = cv2.drawKeypoints(img, keypoints[::10], None, color=(0,255,0), flags=4)
dataset.show_image(img2)

<div class="alert alert-block alert-info">
<b>📝 Упражнение</b>

Просмотрев [документацию по детектору ORB](https://docs.opencv.org/3.4/db/d95/classcv_1_1ORB.html#adc371099dc902a9674bd98936e79739c), попробуйте ихменить некоторые параметры и посмотреть, как это повлияет на обнаружение ключевых точек. Обратите внимание, как показывает опыт, для достаточного описания изображения требуется около 1500 ключевых точек на разных масштабах.

</div>

Вы также можете взять разные изображения и менять реализацию/параметры, чтобы узнать сильные и слабые стороны этого метода

In [ ]:
cbir.marking.this_is_an_action_cell()


# Попробуйте способ выше для случайного изображения в наборе данных
# --- Добавьте код сюда ---
image_id = '110901'
# --- Конец кастомного кода ---


img = dataset.read_image(image_id + '.jpg')

keypoints = find_keypoints(img)

plt.figure(figsize=(15,10))
# Отображаются только каждые 10 ключевых точек
img2 = cv2.drawKeypoints(img, keypoints[::10], None, color=(0,255,0), flags=4)
dataset.show_image(img2)

<br>
<br>

**2.2 Извлечение локальных признаков**  
<hr width=20% align=left>
<div style="margin: auto; float: left; margin-right: 50px; width: 19%">
<img src="images/bow.png">
<em>Мешок визуальных слов <a href="https://towardsdatascience.com/bag-of-visual-words-in-a-nutshell-9ceea97ce0fb">[источник]</a></em>
</div>

В этом разделе мы узнаем, как реализовать такую функцию:
```python
def extract_descriptors(image, keypoints):
    return embedding
```

Чтобы свести все интересные прихнаки изображения, мы реализуем [модель «Мешок визуальных слов»](https://towardsdatascience.com/bag-of-visual-words-in-a-nutshell-9ceea97ce0fb). Этот метод позаимствован из концепции «мешок слов» в области обработки естественных языков и переработан для визуальных элементов.

В процессе CBIR изображение преобразуется в некое проестранство признаков. Это делается для достижения явного сопоставления, чтобы устранить влияние фона и потенциальных преобразований или изменений, но сохранить распознаваемость подлинного визуального содержимого.


Получив признаки изображения, мы реализуем модель «Мешок визуальных слов». Этот метод позаимствован из концепции «мешок слов» в области обработки естественных языков и переработан в визуальный эквивалент.

<div class="alert alert-block alert-info">
<b>📝 Action</b>

Просмотрев [документацию по детектору ORB](https://docs.opencv.org/3.4/db/d95/classcv_1_1ORB.html#adc371099dc902a9674bd98936e79739c), попробуйте реализовать функцию ниже, которая берет изображение и список ключевых точек в качестве входных данных и возвращает вычисленных дескриптор ORB

</div>

In [ ]:
cbir.marking.this_is_an_action_cell()


def extract_descriptors(image, keypoints):
    ''' Эта функция должны взять изображение и ключевые точки в качестве входных данных
    и вернуть список признаков ORB
    '''
    # --- Вставьте свой код сюда ---
    ...
    # --- Конец кастомного кода ---
    return features

<details>
  <summary outline="1pt">🆘 Что-то не получается? Нажмите здесь, чтобы посмотреть возможное решение</summary>

```python
def extract_descriptors(image, keypoints):
    ''' Эта функция должна взять изображение в качестве входных данных
    и вернуть список ключевых точек
    '''
    orb = cv2.ORB.create(1500, nlevels=32)
    keypoints, features = orb.compute(image, keypoints)
    return features
```  
</details>

Попробуйте эту функцию, посмотрев на первые **5** дескрипторов, извлеченных алгоритмом

In [ ]:
img = dataset.read_image('131500.jpg')

keypoints = find_keypoints(img)
features = extract_descriptors(img, keypoints)

print("Дескриптор 1-й ключевой точки:\n {}".format(features[0]))
print("Дескриптор 2-й ключевой точки:\n {}".format(features[1]))
print("Дескриптор 3-й ключевой точки:\n {}".format(features[2]))
print("Дескриптор 4-й ключевой точки:\n {}".format(features[3]))
print("Дескриптор 5-й ключевой точки:\n {}".format(features[4]))

Теперь мы можем извлекать дескрипторы из ключевых точек изображения. Давайте визуализируем некоторые из них

In [20]:
img = dataset.read_image('131200.jpg')
keypoints = find_keypoints(img)
patches = descriptor.extract_patches(img, keypoints)
features = extract_descriptors(img, keypoints)

descriptor.show_random_descriptors(img, keypoints, patches, features)

Конечно, мы можем попробовать то же самое со случайным изображением

In [22]:
img = dataset.get_random_image()

keypoints = find_keypoints(img)
patches = descriptor.extract_patches(img, keypoints)
features = extract_descriptors(img, keypoints)

descriptor.show_random_descriptors(img, keypoints, patches, features)

Вы завершили первую половину урока!

Предлагаем несколько ресурсов, с которыми можно ознакомиться, чтобы узнать больше о технологии контрольных точку и их описании:

1. http://vision.stanford.edu/teaching/cs231a_autumn1112/lecture/lecture11_detectors_descriptors_cs231a.pdf
2. http://vision.stanford.edu/teaching/cs231a_autumn1112/lecture/lecture12_SIFT_single_obj_recog_cs231a.pdf
3. https://docs.opencv.org/3.4/db/d27/tutorial_py_table_of_contents_feature2d.html


Следует особо отметить один важный аспект о ключевых точках, которые использовали в примере. В отличие от ключевых точек Гарриса, которые обычно определяют углы, ключевые точки SIFT также назначают пространственный масштаб: характерные регионы не обязательно маленькие, и могут быть размером со все изображение! Таким образом визуальные отпечатки — хоть они и одного измерения (обычно это 128 элементов) — могут описать всю сцену, но приблизительно. Подробности об отдельных меньших регионах предоставляются дескрипторами, связанными с контрольными точками меньшего пространственного масштаба.

<br>

## **Глава 3: «Создание визуального словаря»**
---

В этом разделе мы узнаем, как реализовать данную функцию:
```python
def create_vocabulary(features):
    # Реализуйте с помощью иерархических k-средних
    # Вохвращает дерево и индекс
```

<br>
    <b>Зачем использовать словарь?</b><br>
Если есть очень большой словарь, то поиск изображений работает очень эффективно. Словарь — это набор всех признаков, извлеченных из доступных данных.

Большой словарь позволяет описать базу данных более точно, что обеспечивает более эффективный поиск.

Тем не менее, большие кодовые словари нереализуемы, поскольку **неиерархиеские методы плохо масштабируются**.
Давайте рассмотрим, почему так происходит.

У нас есть признак ***q***, а также список признаков базы данных ***D***.
<u>Целью является поиск признака ***d<sub>i</sub>*** в базе данных, которая закрыта для признака запроса ***q***.</u>
Рассматриваются два случая:
- Плоский список
- Иерархическое дерево

<br>
<div style="margin:auto; float:left; margin-right: 50px; width: 38%;">
    <center>Случай 1. Плоский список. Можно представить его как дерево со всего <code>l = 1</code> уровнем и <code>k = 27</code> ветвями.</center>
    <center>Чтобы найти ближайший элемент в базе данных следует произвести <code>27</code> сравнений, потому что вы сравниваете признак запроса <i><b>q</b></i> напрямую со всеми 27 признаками из базы данных <i><b>d<sub>i</sub></b></i></center>
    <br>
<img src="images/flat_list.png">
    <center>$k^l$</center>
</div>
<div style="margin:auto; float:right; margin-left: 50px; width: 48%;">
    <center>Случай 1. Дерево с <code>l = 3</code> уровнями и <code>k = 3</code> ветвями.</center>
    <center>Чтобы найти ближайший элемент в базе данных следует произвести <code>9</code> сравнений, потому что вы сравниваете признак запроса <i><b>q</b></i> с первыми <code>k = 3</code> виртуальными элементами, до <code>l = 3</code> уровней.</center>
<img src="images/tree_27.png">
<center>$k * l$</center>
</div>

<br>

Самое важное взятое нами из работы — это **механизм индексирования,**
который обеспечивает крайне эффективный поиск.

В нем предлагается **иерархическая оценка TF-IDF** с помощью
иерархически определенных визуальных слов, которые формируют
словарное дерево.

Это обеспечивает гораздо более эффективный поиск визуальных слов,
что позволяет использовать более крупные словари. Это приводит
к значительному повышению качества поиска.


<div style="margin:auto; float:right; margin-left: 50px; width: 60%">
<!--     <center>Конструкция данного вектора является крайне важной ждя эффективности поиска</b></center> -->

<img src="images/tree_structure.png">
    <center>
    Дерево напрямую определяет визуальный словарь и
    эффективную процедуру поиска в интегрированном
        виде.
    </center>
</div>
<br>
   

В этом разделе мы создадим структуру набора представлений изображений, собранных в разделе **1.2**.  
Мы реализуем [**структуру словарного дерева**](https://ieeexplore.ieee.org/document/1641018), показанную Нистером [источник], в которой для строительства дерева используются иерархические K-средние, а TF-IDF используется для индексирования каждой выборки базы данных.

<br>  

> ***Рецепт:***   
> ***2.1.*** Построение дерева с помощью k-средних   
> ***2.2*** ***Индексирование*** базы данных с помощью TF-IDF (частота слова и обратная частота документа)   
> ***2.3*** ***Оценка*** двух изображений   
> ***2.4*** ***Поиск*** похожих изображений   

<br>

### 3.1 Построение дерева с помощью иерархических k-средних
---

<br>
<div style="margin:auto; float:left; margin-right: 50px; width: 33%;">
<img src="images/kmeans.png">
</div>
<br>

В этом разделе мы иерархически отделим признаки в кластеры, используя ***иерархически k-средние***.

`k` определяет множитель ветви дерева (количество
детей каждого узла). Сначала запускается первый процесс
k-средних на наборе тренировочных данных. Он определяется
центры кластеров k. Тренировочные данные затем делятся
на k-группы, где каждая группа состоит из векторов дескрипторов,
наиболее близких к центру определенного кластера.

Процедура применяется рекурсивно на каждом кластере до достижения глубины `L`.


In [26]:
!dot -c

In [27]:
import cbir
import numpy as np

# инициалзиация базы данных
dataset = cbir.Dataset()
subset = dataset.subset[0:10]
orb = cbir.descriptors.Orb()
voc = cbir.encoders.VocabularyTree(n_branches=2, depth=2, descriptor=orb)

# разделение иерархических k-средних на кластеры по признакам
features = np.array([[1.], [2.], [11.], [12.]])
voc.fit(features)


# построение графа
fig = voc.draw(labels=voc.nodes, figsize=(5, 3), node_color="C1")

- Внутренние узлы дерева являются центрами кластеров
- Листья узлов — это признаки, с которых мы начали



Попробуйте сделать также со своими функциями. Помните:
- `n_branches` контролирует, сколько узлов создавать из родительского узла (`k` на рисунке выше)
- `depth` контролирует количество уровней дерева (`L` на рисунке выше)

In [28]:
n_branches = 2
depth = 3

In [29]:
# инициалзиация базы данных
voc = cbir.encoders.VocabularyTree(n_branches=n_branches, depth=depth, descriptor=orb)

# разделение иерархических k-средних на кластеры по признакам
voc.fit(np.random.randn(150, 1))

# построение графа
fig = voc.draw(figsize=(20, 7), labels=voc.nodes)

<br> 

### 3.2 Индексирование базы данных с помощью TF-IDF (частота слова и обратная частота документа)
---

<div style="margin:auto; float:right; margin-roght: 50px; width: 40%">
<img src="images/index.png">
<!-- <em>Добавьте заголовок изображения со ссылкой</em> -->
</div>
<br>


Индексирование — это процесс построение и хранения **таблицы базы данных** для выполнения эффективного **поиска**, например, для быстрого поиска определенного элемента базы данных.  
Для задачи поиска изображения хранится индекс, который связывает изображение с его векторным представлением: **`image: vector`**

 В этом разделе мы узнаем, как реализуется функция:
```python
def encode(dataset):
    return indexed_database
```

В этом разделе мы создадим структуру набора представлений изображений, собранных в разделе **1.2**.  
Мы реализуем [**структуру словарного дерева**](https://ieeexplore.ieee.org/document/1641018), которая использует инвертированные индексы и иерархические k-средние для построения графа.

***Рецепт:***  
1. При данном изображении для каждого признака мы распространяем признак по дереву.   
2. Начиная с корня дерева находим ближайший узел среди дочерних узлов.  
3. Применяем эту процедуру пока не достигнем дна дерева — листа.  
4. При каждом прохождении через узел регистрируем прохождение, добавляя +1 узлу конкретного изображения.  

<br>
<div style="margin:auto; float:left; margin-right: 50px; width: 42%;">
    <center>
    <b>Признак #1</b>
        <br>
    </center>
<img src="images/encoded_a.png">
</div>
<div style="margin:auto; float:right; margin-left: 50px; width: 42%;">
    <center>
    <b>Признак #2</b>
        <br>
    </center>
<img src="images/encoded_b.png">
</div>
<br>
<br>
<div style="margin:auto; float:left; margin-right: 50px; width: 42%;">
        <center>
            <b>...</b>
            <br>
    <b>Признак #3</b>
    </center>
<img src="images/encoded_c.png">
</div>
<div style="margin:auto; float:right; margin-left: 50px; width: 42%;">
        <center>
            <b>...</b>
            <br>
    <b>Признак #4</b>
    </center>
<img src="images/encoded_d.png">
</div>

<!-- <em>Добавьте заголовок изображения и ссылку</em> -->
</div>
<br>

<div style="margin:auto; float:center; width: 55%;">
    <br>
    <center>
        Мы распространили все признаки и считаем количество прохождений на каждом узле.     
        <b>Это представление закодированного изображения</b>    
    </center>
    <img src="images/encoded_all.png">
</div>
 

В результате получается схема TF-IDF: при пересечении узла мы оставляем на нем отпечаток:

In [ ]:
import matplotlib.pyplot as plt

# рассмотрим как кодируется изображение
image_id = "100000"

# посмотрим, какие мы кодируем изображения
dataset.show_image(image_id)
print("Image as perceived by us:")
plt.show()

# как граф
voc.subgraph(image_id)
print("Image embedding as graph:")
plt.show()

# и соответствующий вектор
embedding = voc.embedding(dataset.read_image(image_id))
print("\nImage embedding as vector:", embedding, "\n")
fig = plt.figure(figsize=(20, 3))
plt.bar(np.arange(len(embedding)), embedding)
plt.gca().set_title("TF-IDF")
plt.show()

<div class="alert alert-block alert-info">
<b>📝 Упражнение</b>

Измените `image_id` выше и попробуйте закодировать другое изображение

</div>

In [ ]:
cbir.marking.this_is_an_action_cell()


# --- Добавьте свой код сюда ---

image_id = "100000"

# --- Конец кастомного кода ---


# --- Построение, не удаляйте код ниже ---
dataset.show_image(image_id)
print("Image as perceived by us:")
plt.show()

# как граф
voc.subgraph(image_id)
print("Image embedding as graph:")
plt.show()

# и соответствующий вектор
embedding = voc.embedding(dataset.read_image(image_id))
print("\nImage embedding as vector:", embedding, "\n")
fig = plt.figure(figsize=(20, 3))
plt.bar(np.arange(len(embedding)), embedding)
plt.gca().set_title("TF-IDF")
plt.show()

<br>

## **Глава 4: «Оценка и поиск (сетевой этап)»**
---


<!-- <div style="margin-left: 230px"> -->
<br>

<div style="margin:auto; float:right; margin-left: 50px; width: 30%">
<img src="images/image_similarity.jpg">
</div>
<br>
    
В этом разделе мы узнаем, как реализовать функцию:
```python
def score(database, image):
    return score
```

Целью оценки является сравнение двух изображений в форме встраивания и возвращение оценки их **подбности**.

Когда определена квантизация, нам нужно определить
значимость изображения из базы данных для изображения из запроса
на основе похожести путей по словарном дереву
для дескрипторов изображения из базы данных
и изображения их запроса.

<br>

  

Возьмем два изображения из базы данных и получим их встраивания

In [ ]:
# возьмите два изображения, используя их id
image_id_1 = "104000"
image_id_2 = "101000"

# прочитайте изображения
image1 = dataset.read_image(image_id_1)
image2 = dataset.read_image(image_id_2)

# затем получите их встраивания
em_1 = voc.embedding(image1)
em_2 = voc.embedding(image2)

print("\nImage 1 embedding:", em_1)
print("\n\nImage 2 embedding:", em_2)

fig, ax = plt.subplots(1, 2, figsize=(30, 3))
ax[0].bar(np.arange(len(em_1)), em_1)
ax[0].set_title("Image 1 TF-IDF")
ax[1].bar(np.arange(len(em_2)), em_2)
_ = ax[1].set_title("Image 2 TF-IDF")

<br>
Мы также может посмотреть на их графовые представления

In [33]:
voc.subgraph(image_id_1)
voc.subgraph(image_id_2)

В этой реализации для измерения оценки между двумя изображениями используется <b>евклидова норма</b>:

$RMSE(x_1, x_2) = \sqrt{\sum(x_{1_i} - x_{2_i})^2}$


In [ ]:
cbir.marking.this_is_an_action_cell()

def score(x1, x2):
    """Возвращает евклидово расстояние между двумя тензорами"""
    # --- Вставьте свой код сюда ---
    rmse = ...
    # --- Конец кастомного кода ---

# --- Построение, не удаляйте код ниже ---
    return rmse

<details>
  <summary outline="1pt">🆘 Что-то не получается? Нажмите здесь, чтобы посмотреть возможное решение</summary>

```python
def score(x1, x2):
    """Возвращает евклидово расстояние между двумя тензорами"""
    # --- Вставьте свой код сюда ---
    rmse = np.sqrt(np.mean(np.square(x1 - x2)))
    # --- Конец кастомного кода ---
```  
</details>

Посмотрим, какая будет оценка между двумя только что выбранными изображениями.    
Если оценивать, насколько похожи изображения (функция перестановочная), мы измеряем оценку между двумя соответствующими встраиваниями.

<div class="alert alert-heading alert-danger" style="background-color: white; border: 2px solid; border-radius: 5px; color: #000; border-color:#AAA; padding: 10px">
<b>📝 Примечание </b>

Посколько используемый признак оценки является формой <b>расстояния </b> между двумя изображениями, чем он меньше, тем лучше.    
Чем ниже оценка, тем ближе изображения!

</div>

In [55]:
# Создадим экземпляр набора данных
dataset = cbir.Dataset()

In [56]:
# Создадим словарное дерево
voc = cbir.encoders.VocabularyTree(n_branches=4, depth=4, descriptor=orb)

In [ ]:
# Теперь извлечем признаки
# На личном компьютере это занимает около 10 минут. 
# Мы уже сделали это для вас и сохранили все признаки в `data/features_orb.hdf5`. Если вы хотите все сделать 
# с нкля, просто удалите или переименуйте этот файл.
features = voc.extract_features(dataset)

In [ ]:
# Теперь мы можем построить дерево, используя извлеченные признаки
voc.fit(features)

In [ ]:
# Извлечем встраивания для двух следующих изображений
image_id_1 = "104000"
image_id_2 = "104002"

# прочитаем изображения
image1 = dataset.read_image(image_id_1)
image2 = dataset.read_image(image_id_2)

In [ ]:
# построение изображения 1
dataset.show_image(image1)

In [ ]:
# построение изображения 2
dataset.show_image(image2)

In [ ]:
# получим их встраивания
em_1 = voc.embedding(image1)
em_2 = voc.embedding(image2)

fig, ax = plt.subplots(2, 1, figsize=(20, 5))
ax[0].bar(np.arange(len(em_1)), em_1)
ax[0].set_title("Image 1 TF-IDF")
ax[1].bar(np.arange(len(em_2)), em_2)
_ = ax[1].set_title("Image 2 TF-IDF")
plt.tight_layout()

In [109]:
s = score(em_1, em_2)
print("The score is:", s)

<br>

### 4.2 Поиск изображения в базе данных
---
Здесь мы объединим все окнцепции, показанные выше, чтобы построить крупномасштабную CBIR-систему.    
Помните, что в прошлом упражнении мы создали кодер:   
эта функция берет изображение и возвращает его <b>представление</b>, встраивая ключевую информацию о его содержимом так, чтобы ее можно было использовать для поиска похожих изображений.

В следующем управжнении мы используем этот кодер, чтобы проиндексировать базу данных для поиска в ней.

In [ ]:
# у нас есть обученный кодер, поэтому давайте создадим базу данных для поиска
db = cbir.Database(dataset, encoder=voc)

In [ ]:
# Можно проиндексировать все изображения
# можно использовать db.index() для одновременной индексации всех изображений
db.index()

# сохраните базу данных на диск на потом
db.save()

In [ ]:
query = "104000.jpg"
scores = db.retrieve(query)
db.show_results(query, scores, figsize=(30, 10))

In [ ]:
import random

query = random.choice(dataset)
scores = db.retrieve(query)
db.show_results(query, scores, figsize=(20, 10))

<br>

## **[Дополнительно] Глава 5. Кодирование изображений с помощью глубинных сверточных нейронных сетей**
---

Упражнения в этом разделе необязательны. Они дают более широкую картину того, как строить эффективное представление изображения.
Мы покажем, как использовать обученный классификатор изображений для представления признаков изображения
- 4.1 Использование вероятностей выхода обученной сети AlexNet для получения встраивания изображения
- 4.2 Использование предпоследнего слоя сверточной нейронной сети в качестве признаков для распространения дерева

**5.1 Использование вероятностей выход обученной сети AlexNet**

В этом дополнительном упражнении мы заменим сопоставление `encode` на вероятности выхода сети *[AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf)*

In [ ]:
# скачаем обученную версию сети AlexNet с torchvision
import torch
import torchvision
import matplotlib.pyplot as plt
import cbir
import numpy as np

In [ ]:
dataset = cbir.Dataset().subset[0:100]
db_ax_enc = cbir.Database(dataset, encoder=cbir.encoders.AlexNet())

In [ ]:
db_ax_enc.index()

In [ ]:
image_path = "104000.jpg"
embedding_ax_enc = db_ax_enc.embedding(image_path)

plt.figure(figsize=(20, 5))
plt.bar(np.arange(len(embedding_ax_enc)), embedding_ax_enc)

In [ ]:
import random

query = random.choice(db_ax_enc.dataset.image_paths)
scores = db_ax_enc.retrieve(query)
db_ax_enc.show_results(query, scores, figsize=(20, 10))

**5.2 Использование функций DCNN**

Мы можем сравнить результаты выше, где мы используем сеть AlexNet в качестве кодера с методом ДЕСКРИПТОР+СЛОВАРЬ, где -- вместо использования сети AlexNet в качестве кодера -- дескрипторы берутся из промежуточного слоя сети AlexNet. Они считаются векторами признаков (несколько для каждого изображения), которые затем кодируются с помощью словарного дерева.

In [ ]:
# создадим словарное дерево, используя дескрипторы, сгенерированные нейронной сетью
voc_nn = cbir.encoders.VocabularyTree(n_branches=4, depth=4, descriptor=cbir.descriptors.AlexNet())

In [ ]:
# Теперь извлечем признаки, используя сеть AlexNet
features = voc_nn.extract_features(dataset)

In [ ]:
# Теперь мы можем построить дерево, используя извлеченные функции
voc_nn.fit(features)

In [ ]:
# Создаем модель как обычно
db_nn = cbir.Database(dataset, encoder=voc_nn)

In [ ]:
# индексируем базу данных,
# то есть рассчитываем встраивания для всех изображений и сохраняем их в базе данных db_nn._database
db_nn.index()

In [ ]:
scores = db_nn.retrieve(query)
db_nn.show_results(query, scores, figsize=(20, 10))